# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 2 2023 2:18PM,256554,10,DNMD-33423,Emerginnova,Released
1,Feb 2 2023 2:18PM,256554,10,Enova-11308,Emerginnova,Released
2,Feb 2 2023 2:17PM,256550,10,0086352490,ISDIN Corporation,Released
3,Feb 2 2023 2:17PM,256550,10,0086352272,ISDIN Corporation,Released
4,Feb 2 2023 2:17PM,256550,10,0086352270,ISDIN Corporation,Released
5,Feb 2 2023 2:17PM,256550,10,0086352275,ISDIN Corporation,Released
6,Feb 2 2023 2:17PM,256550,10,0086352274,ISDIN Corporation,Released
7,Feb 2 2023 2:17PM,256550,10,0086352273,ISDIN Corporation,Released
8,Feb 2 2023 2:17PM,256550,10,0086352279,ISDIN Corporation,Released
9,Feb 2 2023 2:17PM,256550,10,0086352278,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,256550,Released,23
40,256551,Released,3
41,256552,Released,7
42,256553,Released,3
43,256554,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256550,NaN,NaN,23.0
256551,NaN,NaN,3.0
256552,NaN,NaN,7.0
256553,NaN,NaN,3.0
256554,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256440,0.0,4.0,2.0
256444,0.0,4.0,6.0
256454,0.0,1.0,0.0
256456,0.0,0.0,1.0
256466,1.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256440,0,4,2
256444,0,4,6
256454,0,1,0
256456,0,0,1
256466,1,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256440,0,4,2
1,256444,0,4,6
2,256454,0,1,0
3,256456,0,0,1
4,256466,1,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256440,,4,2
1,256444,,4,6
2,256454,,1,
3,256456,,,1
4,256466,1,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 2 2023 2:18PM,256554,10,Emerginnova
2,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation
25,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation
81,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc."
84,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC"
91,Feb 2 2023 2:11PM,256551,10,"Methapharm, Inc."
94,Feb 2 2023 2:04PM,256548,10,Methapharm-G
95,Feb 2 2023 1:39PM,256547,19,"AdvaGen Pharma, Ltd"
96,Feb 2 2023 1:18PM,256545,16,ACG North America LLC
97,Feb 2 2023 1:08PM,256541,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 2 2023 2:18PM,256554,10,Emerginnova,,,2
1,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation,,,23
2,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation,,,56
3,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc.",,,3
4,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC",,,7
5,Feb 2 2023 2:11PM,256551,10,"Methapharm, Inc.",,,3
6,Feb 2 2023 2:04PM,256548,10,Methapharm-G,,,1
7,Feb 2 2023 1:39PM,256547,19,"AdvaGen Pharma, Ltd",,,1
8,Feb 2 2023 1:18PM,256545,16,ACG North America LLC,,,1
9,Feb 2 2023 1:08PM,256541,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 2:18PM,256554,10,Emerginnova,2,,
1,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation,23,,
2,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation,56,,
3,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc.",3,,
4,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC",7,,
5,Feb 2 2023 2:11PM,256551,10,"Methapharm, Inc.",3,,
6,Feb 2 2023 2:04PM,256548,10,Methapharm-G,1,,
7,Feb 2 2023 1:39PM,256547,19,"AdvaGen Pharma, Ltd",1,,
8,Feb 2 2023 1:18PM,256545,16,ACG North America LLC,1,,
9,Feb 2 2023 1:08PM,256541,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 2:18PM,256554,10,Emerginnova,2,,
1,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation,23,,
2,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation,56,,
3,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc.",3,,
4,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC",7,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 2:18PM,256554,10,Emerginnova,2.0,NaN,NaN
1,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation,23.0,NaN,NaN
2,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation,56.0,NaN,NaN
3,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc.",3.0,NaN,NaN
4,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC",7.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 2 2023 2:18PM,256554,10,Emerginnova,2.0,0.0,0.0
1,Feb 2 2023 2:17PM,256550,10,ISDIN Corporation,23.0,0.0,0.0
2,Feb 2 2023 2:17PM,256549,10,ISDIN Corporation,56.0,0.0,0.0
3,Feb 2 2023 2:15PM,256553,10,"Methapharm, Inc.",3.0,0.0,0.0
4,Feb 2 2023 2:13PM,256552,10,"Digital Brands, LLC",7.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3334712,111.0,10.0,1.0
12,1026002,3.0,1.0,0.0
15,769410,17.0,0.0,0.0
16,513079,2.0,0.0,0.0
19,1282571,26.0,33.0,0.0
20,256519,1.0,0.0,0.0
21,2052031,6.0,2.0,0.0
22,769566,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3334712,111.0,10.0,1.0
1,12,1026002,3.0,1.0,0.0
2,15,769410,17.0,0.0,0.0
3,16,513079,2.0,0.0,0.0
4,19,1282571,26.0,33.0,0.0
5,20,256519,1.0,0.0,0.0
6,21,2052031,6.0,2.0,0.0
7,22,769566,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,111.0,10.0,1.0
1,12,3.0,1.0,0.0
2,15,17.0,0.0,0.0
3,16,2.0,0.0,0.0
4,19,26.0,33.0,0.0
5,20,1.0,0.0,0.0
6,21,6.0,2.0,0.0
7,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,111.0
1,12,Released,3.0
2,15,Released,17.0
3,16,Released,2.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,1.0,0.0,0.0,33.0,0.0,2.0,0.0
Released,111.0,3.0,17.0,2.0,26.0,1.0,6.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,0.0,33.0,0.0,2.0,0.0
2,Released,111.0,3.0,17.0,2.0,26.0,1.0,6.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,1.0,0.0,0.0,33.0,0.0,2.0,0.0
2,Released,111.0,3.0,17.0,2.0,26.0,1.0,6.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()